<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/SECTOR/CPU_SECTOR_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 21:48:44] [setup] RAM Tracking...
[codecarbon INFO @ 21:48:44] [setup] GPU Tracking...
[codecarbon INFO @ 21:48:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:48:44] [setup] CPU Tracking...
[codecarbon WARNING @ 21:48:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:48:45] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 21:48:45] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:48:45] >>> Tracker's metadata:
[codecarbon INFO @ 21:48:45]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 21:48:45]   Python version: 3.10.12
[codecarbon INFO @ 21:48:45]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 21:48:45]   Available RAM : 12.675 GB
[codecarbon INFO @ 21:48:45]   CPU count: 2
[codecarbon INFO @ 21:48:45]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:48:45]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

[codecarbon INFO @ 21:49:15] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 21:49:15] Energy consumed for all GPUs : 0.000085 kWh. Total GPU Power : 10.13600295569342 W
[codecarbon INFO @ 21:49:15] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:49:15] 0.000479 kWh of electricity used since the beginning.


# Manual Inputs


In [5]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"



#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "BAAI/bge-base-en-v1.5"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "SECTOR-multilabel-bge"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=7.04e-5   #4.04e-5
batch_size = 16
num_train_epochs= 7
weight_decay=0.095
warmup_steps = 300
gradient_accumulation_steps = 1

# Automated

## Dataset

In [6]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

train_df = train_df.dropna(subset = ['SectorLabel']).reset_index(drop=True)
test_df = test_df.dropna(subset = ['SectorLabel']).reset_index(drop=True)
sector_list = [val  for list_ in list(train_df.SectorLabel) for val in list_]
sector_list = list(set(sector_list))

### Define labels ##################
label_names= sector_list
# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

for sector in sector_list:
    train_df[sector] = train_df.SectorLabel.apply(lambda x: True if sector in x else False)
    test_df[sector] = test_df.SectorLabel.apply(lambda x: True if sector in x else False)

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{0: 'Agriculture', 1: 'Buildings', 2: 'Coastal Zone', 3: 'Cross-Cutting Area', 4: 'Disaster Risk Management (DRM)', 5: 'Economy-wide', 6: 'Education', 7: 'Energy', 8: 'Environment', 9: 'Health', 10: 'Industries', 11: 'LULUCF/Forestry', 12: 'Social Development', 13: 'Tourism', 14: 'Transport', 15: 'Urban', 16: 'Waste', 17: 'Water'}


In [7]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10123
0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207

 Classes Representation in Test Dataset: 936
0 . Agriculture : 200
1 . Buildings : 18
2 . Coastal Zone : 71
3 . Cross-Cutting Area : 180
4 . Disaster Risk Management (DRM) : 85
5 . Economy-wide : 85
6 . Education : 23
7 . Energy : 254
8 . Environment : 91
9 . Health : 68
10 . Industries : 41
11 . LULUCF/Forestry : 193
12 . Social Development : 56
13 . Tourism : 28
14 . Transport : 107
15 . Urban : 51
16 . Waste : 59
17 . Water : 106


In [8]:
# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}

for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))
    pos = sum(train_df[label])
    neg = len(train_df) - sum(train_df[label])
    positive_weights[label] = max(neg/pos, 1.1) if (neg/pos) < 10.0 else 10.0
    negative_weights[label] = min(pos/neg,1.0)
print(positive_weights)
print(negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207
{'Agriculture': 3.5293064876957496, 'Buildings': 10.0, 'Coastal Zone': 10.0, 'Cross-Cutting Area': 4.463032919589854, 'Disaster Risk Management (DRM)': 10.0, 'Economy-wide': 10.0, 'Education': 10.0, 'Energy': 2.5556726378644186, 'Environment': 10.0, 'Health': 10.0, 'Industries': 10.0, 'LULUCF/Forestry': 4.439548629768941, 'Social Development': 10.0, 'Tourism': 10.0, 'Transport': 7.63000852514919, 'Urban': 10.0, 'Waste': 10.0, 'Water': 7.386909693454847}
{'Agriculture': 0.283341784989858, 'Buildings': 0.01697809925658027, 'Coastal Zone': 0.07405835543766578, 'Cross-Cutting Area': 0.22

`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [9]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at BAAI/bge-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 10123
Validation data: 936


Map:   0%|          | 0/10123 [00:00<?, ? examples/s]

[codecarbon INFO @ 21:52:45] Energy consumed for RAM : 0.000317 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 21:52:45] Energy consumed for all GPUs : 0.000749 kWh. Total GPU Power : 27.531368642021032 W
[codecarbon INFO @ 21:52:45] Energy consumed for all CPUs : 0.002834 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:52:45] 0.003899 kWh of electricity used since the beginning.


Map:   0%|          | 0/936 [00:00<?, ? examples/s]

Map:   0%|          | 0/10123 [00:00<?, ? examples/s]

Map:   0%|          | 0/936 [00:00<?, ? examples/s]

## Metrics

In [11]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [12]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [13]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights, reduction='mean')
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Training

In [14]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 21:53:46] [setup] RAM Tracking...
[codecarbon INFO @ 21:53:46] [setup] GPU Tracking...
[codecarbon INFO @ 21:53:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:53:46] [setup] CPU Tracking...
[codecarbon WARNING @ 21:53:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:53:47] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 21:53:47] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:53:47] >>> Tracker's metadata:
[codecarbon INFO @ 21:53:47]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 21:53:47]   Python version: 3.10.12
[codecarbon INFO @ 21:53:47]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 21:53:47]   Available RAM : 12.675 GB
[codecarbon INFO @ 21:53:47]   CPU count: 2
[codecarbon INFO @ 21:53:47]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:53:47]   GPU c

In [ ]:
multi_trainer.train()

In [17]:
tracker.stop()

[codecarbon INFO @ 23:19:45] Energy consumed for RAM : 0.007198 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 23:19:45] Energy consumed for all GPUs : 0.100324 kWh. Total GPU Power : 32.724292819100896 W
[codecarbon INFO @ 23:19:45] Energy consumed for all CPUs : 0.064426 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:19:45] 0.171948 kWh of electricity used since the beginning.


0.06004885270537505

In [18]:
multi_trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

events.out.tfevents.1709675632.0fb3b3ae2e9b.409.0:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/SECTOR-multilabel-bge/commit/65249b8d401719b21b87a60a81e9e0240950bbdc', commit_message='End of training', commit_description='', oid='65249b8d401719b21b87a60a81e9e0240950bbdc', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.7077,4.261288,0.000069,1.0,633,0.548961,0.422615,0.546547,0.495351,0.821096,0.890809,0.821096,0.558020,0.624266,0.597736,25.3457,36.929,2.328
1,0.4546,3.038671,0.000061,2.0,1266,0.500854,0.489851,0.612690,0.520230,0.843823,0.902290,0.843823,0.619863,0.682206,0.636553,25.2837,37.020,2.334
2,0.3105,2.656664,0.000047,3.0,1899,0.494661,0.500514,0.659348,0.531714,0.850816,0.896960,0.850816,0.630261,0.712480,0.647437,25.0722,37.332,2.353
3,0.2044,3.219459,0.000031,4.0,2532,0.543048,0.575745,0.704409,0.597040,0.810606,0.880118,0.810606,0.673282,0.737924,0.683428,25.6694,36.464,2.298
4,0.1314,1.835635,0.000015,5.0,3165,0.563265,0.613203,0.738542,0.627112,0.806527,0.877170,0.806527,0.696703,0.760551,0.703183,25.6766,36.453,2.298
5,0.0892,1.457881,0.000004,6.0,3798,0.607322,0.642482,0.749884,0.654501,0.784382,0.860989,0.784382,0.706376,0.763438,0.711254,25.3483,36.926,2.328
6,0.0721,1.118793,0.000000,7.0,4431,0.611372,0.642823,0.748754,0.651869,0.785548,0.862742,0.785548,0.707055,0.763806,0.710948,25.4330,36.803,2.320


# Predict and Evaluate

## Trainer

In [19]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()
y_true = y_true.bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

Agriculture
[[677  59]
 [ 32 168]] 

Buildings
[[905  13]
 [  3  15]] 

Coastal Zone
[[828  37]
 [ 20  51]] 

Cross-Cutting Area
[[648 108]
 [ 47 133]] 

Disaster Risk Management (DRM)
[[817  34]
 [ 24  61]] 

Economy-wide
[[775  76]
 [ 34  51]] 

Education
[[905   8]
 [  8  15]] 

Energy
[[617  65]
 [ 35 219]] 

Environment
[[787  58]
 [ 23  68]] 

Health
[[849  19]
 [ 13  55]] 

Industries
[[876  19]
 [  6  35]] 

LULUCF/Forestry
[[682  61]
 [ 29 164]] 

Social Development
[[858  22]
 [ 13  43]] 

Tourism
[[896  12]
 [ 11  17]] 

Transport
[[794  35]
 [ 19  88]] 

Urban
[[844  41]
 [ 22  29]] 

Waste
[[850  27]
 [  6  53]] 

Water
[[775  55]
 [ 23  83]] 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.740088,0.840000,0.786885,200.0
1,Buildings,0.535714,0.833333,0.652174,18.0
2,Coastal Zone,0.579545,0.718310,0.641509,71.0
3,Cross-Cutting Area,0.551867,0.738889,0.631829,180.0
4,Disaster Risk Management (DRM),0.642105,0.717647,0.677778,85.0
5,Economy-wide,0.401575,0.600000,0.481132,85.0
6,Education,0.652174,0.652174,0.652174,23.0
7,Energy,0.771127,0.862205,0.814126,254.0
8,Environment,0.539683,0.747253,0.626728,91.0
9,Health,0.743243,0.808824,0.774648,68.0
